In [1]:
import os
import sys 
!pip install --user numpy
import numpy as nd 
print(os.getcwd())
#!pip install --user nba_api
#!pip install --user pandas
#!pip install --user requests
print(sys.version)
print(sys.executable)
#!{sys.executable} -m pip install --user nba_api


C:\Users\Temesgen\Documents\GitHub\NBASTATS
3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
C:\Users\Temesgen\anaconda3\python.exe


In [2]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [3]:


import nba_api
import pandas as pd
import requests
import time
from IPython.display import display, HTML
from nba_api.stats.static import teams

from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playbyplayv2

pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [4]:
#Get information about playyers

#nba_players = players.get_players()
#df = pd.DataFrame(nba_players)
#df.sample(5)



active_players = players.get_active_players()
df = pd.DataFrame(active_players)
df.sample(10)

active_players = players.get_active_players()
df = pd.DataFrame(active_players)

booker = df.loc[(df["last_name"] == "Booker")]
booker

,id,full_name,first_name,last_name,is_active
50,1626164,Devin Booker,Devin,Booker,True


In [5]:
#Get Team Information
nba_teams = teams.get_teams()
allteams = pd.DataFrame(nba_teams)


suns = teams.find_team_by_abbreviation('phx')
#suns


# get game logs from the reg season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable="1610612756",season_type_nullable='Regular Season')
#print(type(gamefinder))
games = gamefinder.get_data_frames()[0]
game_ids = games["GAME_ID"].to_list()

f = open("demofile3.txt", "w")

#for game in game_ids:
#    pbp = playbyplayv2.PlayByPlayV2(game)
#    pbp = pbp.get_data_frames()[0]
#    f.write(pbp.to_string())
#    time.sleep(1)
f.close()


In [6]:
def logRuns(pbp, team):
    #f = open("demofile3.txt", "w")
    #g = open("demofile4.txt", "w")
    timeout = (team.upper() + " Timeout")
    
    #g.write(pbp.to_string())
    #g.close()
  
    
    btat = []
    #print(team.upper() + " Timeout" + pbp.iloc[47]['VISITORDESCRIPTION'])
    
    pbpf = pbp.loc[(pbp['SCORE'].notna()) |  (pbp['HOMEDESCRIPTION'].str.contains(team.upper() + " Timeout"))  |  (pbp['VISITORDESCRIPTION'].str.contains(team.upper() + " Timeout")) 
                  |  (pbp['HOMEDESCRIPTION'].str.contains(team + " Timeout"))  |  (pbp['VISITORDESCRIPTION'].str.contains(team + " Timeout"))]
    pbpf = pbpf.reset_index(drop=True)
    #f.write(pbpf.to_string())
    #f.close()
    
    
    for index,play in pbpf.iterrows():
        #print(str(index))
                                              
        playHD = str(play["HOMEDESCRIPTION"]).upper()
        vistHD = str(play["VISITORDESCRIPTION"]).upper()
        
        #print(playHD)
        #print(vistHD)
        if timeout.upper() in playHD or timeout.upper() in vistHD:

            #was there a run?
                        
            
            if index > 10 and index + 10 < len(pbpf) : #make sure there are enough plays left in the game
                #print(str(index))
            
                for i in range(0,5):
                    if  pbpf.iloc[index -(11 + i)]['SCORE'] != None:
                        startscore = pbpf.iloc[index -(11 + i)]['SCORE'].split(" - ")
              
                if  pbpf.iloc[index -1]['SCORE'] == None:
                    break
                
                endscore = pbpf.iloc[index -1]['SCORE'].split(" - ")
                
                if timeout.upper() in vistHD:
                    homediff = int(endscore[1])-int(startscore[1])
                    visitdiff = int(endscore[0])-int(startscore[0])
                    bt_diff = visitdiff -homediff
                else:                     
                    visitdiff = int(endscore[1])-int(startscore[1])
                    homediff = int(endscore[0])-int(startscore[0])
                    bt_diff = visitdiff -homediff
                
                #print(str(visitdiff -homediff))
                #print(startscore)
                #print(endscore)
                
                if index + 15 < len(pbpf):

                    startscore = pbpf.iloc[index - 1]['SCORE'].split(" - ")
                    
                    for i in range(0,5):
                        if  pbpf.iloc[index + (10 + i)]['SCORE'] != None:
                            endscore = pbpf.iloc[index + (10 + i)]['SCORE'].split(" - ")
                    
                    if timeout.upper() in vistHD:
                        visitdiff = int(endscore[1])-int(startscore[1])
                        homediff = int(endscore[0])-int(startscore[0])
                        at_diff = visitdiff -homediff
                    else:
                        homediff = int(endscore[1])-int(startscore[1])
                        visitdiff = int(endscore[0])-int(startscore[0])
                        at_diff = visitdiff -homediff

                    #print(str(visitdiff -homediff))
                    #print(startscore)
                    #print(endscore)
                    btat.append([bt_diff, at_diff])
        
    return btat


game = game_ids[81]
pbp = playbyplayv2.PlayByPlayV2("0022200343")
pbp = pbp.get_data_frames()[0]

print("[Before timeout, After timeout]")
print(logRuns(pbp, "Jazz"))

print(allteams.head(1))

[Before timeout, After timeout]
[[-2, 2], [2, 0], [-3, -2]]
           id      full_name abbreviation nickname     city    state  \
0  1610612737  Atlanta Hawks          ATL    Hawks  Atlanta  Georgia   

   year_founded  
0          1949  


In [ ]:
#for each team pull all their cames and pull the run for each game
    #teams["id"]
    #teams["nickname"]
    
#gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable="1610612737",season_type_nullable='Regular Season')
#print(type(gamefinder))
#games = gamefinder.get_data_frames()[0]
#game_ids = games["GAME_ID"].to_list()
#pbp = playbyplayv2.PlayByPlayV2(game)
#pbp = pbp.get_data_frames()[0]
import numpy as np
allruns = {}



for index, teams in allteams.iterrows():
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable=teams["id"],season_type_nullable='Regular Season')
    games = gamefinder.get_data_frames()[0]
    game_ids = games["GAME_ID"].to_list()
    print(teams["nickname"])
    allruns[teams["nickname"]] = []
    for game in game_ids:
        print(game)
        pbp = playbyplayv2.PlayByPlayV2(game)
        pbp = pbp.get_data_frames()[0]
        time.sleep(.3)
        runs  = logRuns(pbp, teams["nickname"])
        if len(runs) > 0:
            #print(runs)
            allruns[teams["nickname"]] .append(runs)
    print(allruns)
    ##print(allruns)
        
np.save('allruns.npy', allruns)


    

Hawks
0022201216
0022201205
0022201191
0022201182
0022201167
0022201153
0022201131
0022201115
0022201107
0022201090
0022201078
0022201065
0022201048
0022201021
0022201008
0022200999
0022200985
0022200974
0022200958
0022200947
0022200923
0022200911
0022200896
0022200876
0022200857
0022200850
0022200832
0022200818
0022200801
0022200792
0022200777
0022200763
0022200745
0022200725
0022200708
0022200694
0022200684
0022200670
0022200661
0022200644
0022200633
0022200620
0022200604
0022200590
0022200575
0022200558
0022200530
0022200517
0022200508
0022200481
0022200468
0022200454
0022200428
0022200416
0022200404
0022200400
0022200380
0022200365
0022200354
0022200327
0022200313
0022200299
0022200293
0022200278
0022200263
0022200248
0022200235
0022200214
0022200204
0022200188
0022200173
0022200162
0022200149
0022200134
0022200110
0022200099
0022200085
0022200070
0022200057
0022200038
0022200020
0022200005
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1], [

Cavaliers
0022201218
0022201197
0022201178
0022201173
0022201154
0022201131
0022201117
0022201094
0022201079
0022201049
0022201035
0022201027
0022201015
0022201000
0022200987
0022200971
0022200957
0022200935
0022200914
0022200896
0022200886
0022200875
0022200858
0022200852
0022200842
0022200822
0022200809
0022200803
0022200779
0022200764
0022200754
0022200739
0022200733
0022200715
0022200695
0022200685
0022200672
0022200658
0022200645
0022200631
0022200614
0022200603
0022200588
0022200565
0022200552
0022200540
0022200524
0022200498
0022200484
0022200466
0022200452
0022200440
0022200431
0022200421
0022200406
0022200390
0022200381
0022200360
0022200348
0022200330
0022200312
0022200303
0022200295
0022200280
0022200261
0022200248
0022200244
0022200225
0022200216
0022200193
0022200182
0022200171
0022200158
0022200140
0022200120
0022200107
0022200090
0022200072
0022200056
0022200039
0022200032
0022200008
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1

Pelicans
0022201225
0022201209
0022201194
0022201185
0022201162
0022201147
0022201135
0022201127
0022201112
0022201095
0022201080
0022201067
0022201051
0022201030
0022201017
0022201011
0022200988
0022200976
0022200953
0022200941
0022200922
0022200906
0022200890
0022200882
0022200864
0022200842
0022200818
0022200807
0022200798
0022200782
0022200768
0022200755
0022200747
0022200724
0022200716
0022200699
0022200683
0022200673
0022200658
0022200632
0022200621
0022200605
0022200593
0022200586
0022200572
0022200554
0022200543
0022200534
0022200520
0022200502
0022200488
0022200477
0022200457
0022200444
0022200426
0022200412
0022200395
0022200384
0022200371
0022200344
0022200333
0022200318
0022200304
0022200279
0022200269
0022200253
0022200231
0022200217
0022200206
0022200191
0022200175
0022200166
0022200147
0022200134
0022200128
0022200117
0022200088
0022200079
0022200053
0022200040
0022200017
0022200006
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1]

Bulls
0022201223
0022201210
0022201193
0022201182
0022201165
0022201148
0022201140
0022201129
0022201114
0022201104
0022201087
0022201072
0022201060
0022201050
0022201037
0022201009
0022200989
0022200963
0022200950
0022200934
0022200925
0022200912
0022200900
0022200883
0022200874
0022200863
0022200852
0022200833
0022200819
0022200811
0022200796
0022200781
0022200766
0022200744
0022200732
0022200713
0022200708
0022200678
0022200650
0022200636
0022200619
0022200606
0022200592
0022200581
0022200570
0022200552
0022200540
0022200532
0022200519
0022200501
0022200485
0022200468
0022200462
0022200449
0022200434
0022200418
0022200400
0022200391
0022200367
0022200350
0022200336
0022200321
0022200306
0022200281
0022200267
0022200250
0022200227
0022200217
0022200196
0022200166
0022200151
0022200141
0022200124
0022200111
0022200103
0022200084
0022200077
0022200061
0022200047
0022200032
0022200019
0022200007
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1], [

Mavericks
0022201224
0022201210
0022201195
0022201167
0022201161
0022201139
0022201123
0022201113
0022201101
0022201086
0022201075
0022201054
0022201040
0022201023
0022201010
0022200988
0022200983
0022200962
0022200943
0022200929
0022200913
0022200891
0022200881
0022200865
0022200854
0022200844
0022200829
0022200813
0022200800
0022200782
0022200761
0022200750
0022200734
0022200717
0022200701
0022200686
0022200670
0022200654
0022200647
0022200630
0022200617
0022200602
0022200593
0022200577
0022200556
0022200542
0022200528
0022200512
0022200494
0022200487
0022200473
0022200456
0022200440
0022200436
0022200421
0022200405
0022200391
0022200386
0022200362
0022200358
0022200337
0022200325
0022200310
0022200298
0022200286
0022200264
0022200246
0022200230
0022200218
0022200207
0022200190
0022200172
0022200161
0022200155
0022200127
0022200115
0022200093
0022200086
0022200066
0022200053
0022200034
0022200013
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1

...

76ers
0022201217
0022201205
0022201198
0022201181
0022201174
0022201150
0022201139
0022201126
0022201110
0022201103
0022201087
0022201072
0022201057
0022201047
0022201035
0022201016
0022200998
0022200980
0022200972
0022200960
0022200943
0022200936
0022200920
0022200908
0022200889
0022200875
0022200860
0022200846
0022200837
0022200824
0022200804
0022200791
0022200770
0022200756
0022200741
0022200721
0022200698
0022200682
0022200668
0022200655
0022200646
0022200626
0022200612
0022200596
0022200581
0022200567
0022200554
0022200545
0022200534
0022200506
0022200493
0022200480
0022200467
0022200453
0022200432
0022200409
0022200399
0022200382
0022200356
0022200332
0022200312
0022200299
0022200297
0022200273
0022200260
0022200256
0022200237
0022200226
0022200195
0022200188
0022200173
0022200148
0022200122
0022200108
0022200097
0022200084
0022200073
0022200059
0022200044
0022200028
0022200015
0022200001
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1], [

Suns
0022201229
0022201212
0022201201
0022201188
0022201172
0022201160
0022201144
0022201125
0022201110
0022201105
0022201092
0022201064
0022201046
0022201033
0022201025
0022201013
0022200990
0022200962
0022200950
0022200933
0022200910
0022200899
0022200885
0022200870
0022200836
0022200832
0022200817
0022200794
0022200788
0022200777
0022200762
0022200748
0022200734
0022200718
0022200703
0022200697
0022200681
0022200663
0022200637
0022200624
0022200615
0022200603
0022200589
0022200565
0022200550
0022200531
0022200516
0022200510
0022200497
0022200490
0022200464
0022200459
0022200444
0022200427
0022200411
0022200395
0022200384
0022200373
0022200358
0022200345
0022200334
0022200321
0022200307
0022200289
0022200283
0022200259
0022200240
0022200232
0022200220
0022200202
0022200177
0022200167
0022200148
0022200139
0022200130
0022200106
0022200094
0022200079
0022200055
0022200043
0022200027
0022200013
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1], [-

Trail Blazers
0022201230
0022201215
0022201199
0022201184
0022201166
0022201159
0022201145
0022201127
0022201120
0022201104
0022201091
0022201069
0022201053
0022201034
0022201017
0022200998
0022200986
0022200977
0022200966
0022200947
0022200941
0022200931
0022200917
0022200894
0022200872
0022200867
0022200843
0022200830
0022200815
0022200796
0022200787
0022200771
0022200763
0022200751
0022200727
0022200711
0022200705
0022200682
0022200667
0022200654
0022200647
0022200631
0022200616
0022200597
0022200580
0022200571
0022200559
0022200536
0022200504
0022200489
0022200474
0022200458
0022200441
0022200436
0022200420
0022200407
0022200394
0022200376
0022200351
0022200343
0022200324
0022200311
0022200290
0022200274
0022200261
0022200251
0022200238
0022200221
0022200209
0022200190
0022200175
0022200159
0022200150
0022200139
0022200130
0022200116
0022200080
0022200065
0022200051
0022200037
0022200027
0022200014
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-

Kings
0022201227
0022201211
0022201195
0022201185
0022201170
0022201159
0022201145
0022201128
0022201111
0022201105
0022201081
0022201076
0022201059
0022201043
0022201037
0022201026
0022201013
0022200997
0022200976
0022200961
0022200954
0022200928
0022200915
0022200902
0022200894
0022200870
0022200854
0022200844
0022200827
0022200812
0022200807
0022200786
0022200775
0022200758
0022200746
0022200728
0022200712
0022200698
0022200691
0022200677
0022200652
0022200640
0022200625
0022200610
0022200595
0022200575
0022200563
0022200547
0022200537
0022200522
0022200514
0022200491
0022200475
0022200460
0022200429
0022200417
0022200409
0022200397
0022200381
0022200369
0022200350
0022200338
0022200323
0022200307
0022200275
0022200263
0022200257
0022200242
0022200222
0022200210
0022200197
0022200183
0022200171
0022200156
0022200132
0022200109
0022200096
0022200081
0022200069
0022200042
0022200036
0022200014
{'Hawks': [[[1, -7], [4, 14], [-7, -2], [-8, 0], [-2, -2]], [[-10, -14], [8, -1], [-1, 1], [

Spurs
0022201224
0022201214
0022201199
0022201188
0022201170
0022201158
0022201143
0022201116
0022201098
0022201089
0022201080
0022201065
0022201052
0022201040
0022201032
0022201018
0022201002
0022200967
0022200959
0022200944
0022200930
0022200909
0022200891
0022200873
0022200858
0022200850
0022200835
0022200826
0022200811
0022200791
0022200775
0022200760
0022200748
0022200735
0022200729
0022200711
0022200687
0022200666
0022200652
0022200635
0022200623
0022200608
0022200591
0022200585
0022200568
0022200555
0022200542
0022200527
0022200511
0022200503
0022200479
0022200477
0022200455
0022200439
0022200420
0022200406
0022200387
0022200375
0022200345
0022200333
0022200319
0022200288
0022200282
0022200269
0022200247
0022200239
